In [3]:
import pandas as pd
from dbfread import DBF
import numpy as np

In [ ]:
path = 'D:\\IPMIProject\\master test\\20231223.dbf' # 文件目录
table = DBF(path, encoding='GBK')
df = pd.DataFrame(iter(table))
print(df)

In [13]:
# 数据预处理尝试

allowed_values1 = ['101','109','199']
allowed_values2 = ['201','202','203','204']
allowed_values3 = ['301','302','303','306','307','311','312','313','314','315','333','396','397','398']
allowed_values4 = ['408','414','415','497','498']
df.loc[~df['ZZLLM'].isin(allowed_values1), 'ZZLLM'] = '000'
df.loc[~df['WGYM'].isin(allowed_values2), 'WGYM'] = '000'
df.loc[~df['YWK1M'].isin(allowed_values3), 'YWK1M'] = '000'
df.loc[~df['YWK2M'].isin(allowed_values4), 'YWK2M'] = '000'
df.columns = df.columns.map(str.lower)
arr = df
# print(arr.head(10))

# ["bmh","zzllm","wgym","ywk1m","ywk2m","bkdwdm"]
# 取出dataframe的几列
df1 = arr[["bmh","zzllm","wgym","ywk1m","ywk2m","bkdwdm"]]

# 使用其中的四个属性进行分组
df2 = df1.groupby(["zzllm","wgym","ywk1m","ywk2m"]).count()
df2.to_excel("classfier.xlsx")


# sub_DF=[arr[arr['wgymc'].isin([i])] for i in np.unique(arr['wgymc'])]
# print(df2.head())
# groupby后的数据要先转为dataframe，再进行索引重建，才可以保存为xlsx
# class_arr = pd.DataFrame(df1.groupby(["zzllmc","wgymc","ywk1mc","ywk2mc"]))
# class_arr.reset_index(inplace=True)
# class_arr.to_excel("res.xlsx")

In [14]:
# 按照外国语考点代码拆分数据

import numpy as np
import pandas as pd

# arr = pd.read_excel("tmp.xlsx")
list1 = arr['wgykddm'].unique()
groups = arr.groupby("wgykddm")
for i in arr['wgykddm'].unique():
    groups.get_group(i).to_excel(str(i)+".xlsx")
# print(groups['wgykddm'])
    # 检查数量是否相等
# print(arr.shape)#(16383,32)
# sum = 0;
# for i in arr['wgykddm'].unique():
#     df1 = pd.read_excel(str(i)+".xlsx")
#     sum += df1.shape[0]

# print(sum)#16383

In [16]:
# 统计每个考点有多少专业，以及每个专业有多少人，参考属性有["zzllmc","wgymc","ywk1mc","ywk2mc","bkdwdm"]

for i in arr['wgykddm'].unique():
    df1 = pd.read_excel(str(i)+".xlsx")

    # 使用其中的四个属性进行分组,"bkdwdm"
    df2 = df1.groupby(["zzllm","wgym","ywk1m","ywk2m"]).count()
    df2.to_excel("classfier"+str(i)+".xlsx")

# 结果里面bmh表示的是每个专业的人数

In [21]:
# 数据预处理、提取出每个考点，每个专业的人数
sums = 0
print(list1)
for i in list1:
    # .fillna(method="ffill")向下填充
    res = pd.read_excel("classfier"+str(i)+".xlsx").fillna(method="ffill")
    # dataframe按各专业人数一列排序，inplace原地排序，ascending升序
    # res.sort_values(by="bmh",inplace=True,ascending=False)
    # 取出各专业人数一列转为array
    # 若存在控制需要填充
    # res = res.fillna(res['bmh'].mean())
    StuNum = res[['zzllmc','wgymc','ywk1mc','ywk2mc','bmh']]
    np.savetxt("classfier"+str(i)+".txt",StuNum,fmt="%s,%s,%s,%s,%s",delimiter=" ")
    sums += res['bmh'].sum()

print(sums)

# classfierX.excel中的前四列可以映射到对应的专业

['6116' '6152' '6103' '6107' '6146' '6144' '6143' '6139' '6117' '6153'
 '6118' '6160' '6141' '6114' '6113' '6124' '6134' '6105' '6136' '6149'
 '6159' '6157' '6151' '6115' '6112' '6111' '6156' '6133' '6148' '6140'
 '6125' '6154' '6147' '6150' '6145' '6127' '6138' '6128' '6122' '6129'
 '6130' '6131' '6142' '6158']
172888


In [1]:
def Sum_matrix(matrix):
    sum=0
    for i in range(len(matrix)):
        for j in range(len(matrix[i])):
            sum+=matrix[i][j]
    return sum

In [4]:
# 统计每个考点教室所能容纳的人数
groups1 = pd.read_excel("考场容量表.xls").groupby("zzllkddm")
print(groups1.apply(lambda x:x))
print(groups1.get_group(6103)["renshu"].tolist())

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
*** No CODEPAGE record, no encoding_override: will use 'iso-8859-1'
      zzllkddm zzllkcbh  renshu
0         6103        1      30
1         6103       10      30
2         6103      100      30
3         6103      101      30
4         6103      102      30
...        ...      ...     ...
5420      6160      103      30
5421      6160      104      30
5422      6160      105      30
5423      6160      106      30
5424      6160      107      16

[5425 rows x 3 columns]
[30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 29, 30, 30, 30, 31, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 26, 3

In [5]:
import pandas as pd
pd.options.display.max_rows = None
# 分组算法
# pd.set_option('display.max_rows',200)
StuNum = pd.read_excel("classfier6111.xlsx")
# print(StuNum.shape[0])
# aa=pd.read_excel("classfier6105.xlsx").iloc[:,:5]
# aa.columns=['s1','s2','s3','s4','num']
# print(aa.sort_values(by='num',axis=0).reset_index(drop=True))
# 对txt文件的数据排序，方便后面索引对应
# print("各类别人数")
# 统计考点各考场容量
max_student=groups1.get_group(6111)["renshu"].tolist()
print(max_student)
for i in range(len(max_student)):
    if(max_student[i]<30):
        max_student[i]=30
# print("考场容量：",sum(max_student))
print("考场数量：",max_student)
# print("考生总数：",sum(StuNum.iloc[:,4]))
# print(StuNum)
# print(StuNum.iloc[4,4])
for i in range(0,StuNum.shape[0]):
    # print(StuNum.iloc[i,4])
    while StuNum.iloc[i,4] >= 30:
        StuNum.iloc[i,4] = StuNum.iloc[i,4] - max_student[0]
        if StuNum.iloc[i,4] < 0:
            StuNum.iloc[i,4] = 0
        del max_student[0]
print("剩余教室数量：",len(max_student))
# print(max_student)

# 设置参数：教室最多可容纳人数、考场允许最大专业数量
max_subject=5
# 存放分组结果
groups = []
roomV = []
stusum = []

# 这里还没有考虑取余的情况

StuNum.sort_values(by="Unnamed: 0" , inplace=True, ascending=True)
print(StuNum.iloc[:,:5])
StuNum = StuNum.iloc[:,4].values
start = 0
end = len(StuNum)-1
classroom = 0
class_sum = 0
while(start<=end and classroom<len(max_student)):
    class_subject = []#用来存储人数对应的索引，输出的就是这个，和上面txt文件的输出对应
    class_sum = StuNum[end]#每次循环初始，让sum等于尾指针指向的数
    subject = 1
    class_subject.append(end)
    # 人数不超过教室容量、专业不超过教室容量、专业还未分完
    while(class_sum+StuNum[start]<=max_student[classroom] and subject<max_subject and start<=end):
        class_sum = class_sum + StuNum[start]
        class_subject.append(start)
        start = start+1
        subject = subject+1
    print(class_sum,max_student[classroom])
    end = end-1
    stusum.append(class_sum)
    roomV.append(max_student[classroom])
    classroom+=1
    groups.append(class_subject)

print("专业序号",groups)
print("教室容量",roomV)
print("学生实际数量：",stusum)
# 下面的数字是不同袋子的编号

[32, 32, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 32, 35, 35, 35, 35, 35, 32, 36, 36, 20, 20, 32, 20, 20, 35, 30, 40, 40, 40, 35, 35, 40, 32, 35, 20, 35, 20, 30, 30, 40, 35, 35, 30, 32, 30, 35, 40, 40, 20, 35, 20, 30, 20, 40, 32, 35, 30, 20, 20, 35, 20, 30, 20, 20, 35, 32, 20, 35, 35, 35, 40, 40, 30, 40, 35, 40, 31, 42, 28, 28, 28, 28, 28, 28, 28, 28, 28, 30, 28, 28, 28, 28, 28, 40, 35, 35, 40, 30, 30, 40, 20, 40, 35, 35, 35, 30, 35, 35, 30, 32, 30, 30, 35, 35, 40, 40, 30, 30, 30, 35, 30, 30, 35, 35, 35, 40, 30, 35, 40, 40, 35, 40, 30, 30, 30, 30, 30, 34, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 21, 30, 30, 30, 30, 30, 32, 30, 30, 30, 30, 30, 30, 30, 45, 45, 45, 32, 45, 45, 45, 45, 45, 42, 40, 40, 40, 30, 32, 30, 30, 30, 30, 30, 30, 30, 40, 30, 40, 32, 40, 35, 35, 35, 35, 40, 40, 35, 35, 35, 32, 35, 40, 40, 35, 35, 35, 30, 30, 35, 35, 32, 35, 35, 40, 35, 35, 35, 35, 35, 35, 35, 32, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35]
考场数量： [32, 32, 35, 35, 35, 35, 35, 35, 

In [15]:
# 分组算法
import os
list1 = []
names = os.listdir("./")
for name in names:
    if name.endswith('.txt'):
        list1.append(name)
print(list1)

summ=0
for j in list1:
    # 读取文件并定位到人数一列
    if j=='classfier6127.txt':
        print(j)
        # header=None表示没有列标签
    StuNum = pd.read_csv(j,encoding='gb2312',header=None).iloc[:,0]
    # print(StuNum)

    # 设置参数：教室最多可容纳人数、考场允许最大专业数量
    max_student=30
    max_subject=5
    groups = []
    dic = {}
    # 建立原始值 和 余数的映射
    for i in range(len(StuNum)):
        dic[StuNum[i]] = StuNum[i] % 30

    # 保存字典映射关系
    StuNum = []
    with open('params.txt', 'w') as f:
        for key, value in dic.items():
            f.write(str(key))
            f.write(': ')
            f.write(str(value))
            f.write('\n')
            StuNum.append(value)


    # 对余数进行排序
    # dic = sorted(dic.items(),key=lambda x:x[1],reverse=False)
    StuNum.sort()
    # print(len(StuNum))
    start = 0
    end = len(StuNum)-1
    while(start<end):
        class_subject = []
        class_sum = StuNum[end]
        subject = 1
        class_subject.append(end)
        # ===================================================文件6127在这里出了问题
        while(class_sum<max_student and subject<max_subject):
            class_sum = class_sum + StuNum[start]
            class_subject.append(start)
            start = start+1
            subject = subject+1
        end = end-1
        groups.append(class_subject)

    print(groups)
    print(j)
    summ=summ+Sum_matrix(groups)
    print("\n")
print(summ)
    # 下面的数字是不同袋子的编号

['classfier6103.txt', 'classfier6105.txt', 'classfier6107.txt', 'classfier6111.txt', 'classfier6112.txt', 'classfier6113.txt', 'classfier6114.txt', 'classfier6115.txt', 'classfier6116.txt', 'classfier6117.txt', 'classfier6118.txt', 'classfier6122.txt', 'classfier6124.txt', 'classfier6125.txt', 'classfier6127.txt', 'classfier6128.txt', 'classfier6129.txt', 'classfier6130.txt', 'classfier6131.txt', 'classfier6133.txt', 'classfier6134.txt', 'classfier6136.txt', 'classfier6138.txt', 'classfier6139.txt', 'classfier6140.txt', 'classfier6141.txt', 'classfier6142.txt', 'classfier6143.txt', 'classfier6144.txt', 'classfier6145.txt', 'classfier6146.txt', 'classfier6147.txt', 'classfier6148.txt', 'classfier6149.txt', 'classfier6150.txt', 'classfier6151.txt', 'classfier6152.txt', 'classfier6153.txt', 'classfier6154.txt', 'classfier6156.txt', 'classfier6157.txt', 'classfier6158.txt', 'classfier6159.txt', 'classfier6160.txt', 'params.txt']


ParserError: Error tokenizing data. C error: Expected 6 fields in line 804, saw 8
